## 1. Obtendo dados cadastrais de empresas de capital aberto

In [ ]:
import pandas as pd
import numpy as np
import requests

In [ ]:
link = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv'

In [ ]:
r = requests.get(link)

In [ ]:
# gerando uma lista a partir do arquivo usando um for 
linhas = []
# for i in r.text.split('\n'):
#  linhas.append(i.strip().split(';'))

# O mesmo pode ser feito usando list comprehension, uma sintaxe do Python que permite gerar a lista de forma mais eficiente e com sintaxe reduzida 
linhas = [i.strip().split(';') for i in r.text.split('\n') ]


In [ ]:
linhas

In [ ]:
# Gerando um DataFrame usando a função DataFrame do pandas
df = pd.DataFrame(linhas[1:],columns = linhas[0])

In [ ]:
df[df.DENOM_SOCIAL.str.contains('ALIANSCE', na=False)]

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# Filtrando emrpesas de um setor específico
searchfor = ['MULTIPLAN', 'MALL', 'IGUATEMI', 'SHOPP']
shoppings = df[df.DENOM_SOCIAL.str.contains('|'.join(searchfor),na = False)]

In [ ]:
shoppings

In [ ]:
shoppings_ativos = shoppings[shoppings.SIT == 'ATIVO'] 

# shoppings = df[df.DENOM_SOCIAL.str.contains('|'.join(searchfor),na = False)]

In [ ]:
shoppings_ativos.shape

In [ ]:
shoppings_ativos

In [ ]:
list(shoppings_ativos['CD_CVM'])

### Filtrando por CNPJ

In [ ]:
# mostrar todas as colunas do dataframe
pd.set_option('display.max_columns', None)

In [ ]:
# lista_cnpjs = ['05.878.397/0001-32','08.294.224/0001-65', '07.816.890/0001-53', '51.218.147/0001-93', '06.977.751/0001-49']
lista_cnpjs = ['05.878.397/0001-32', '07.816.890/0001-53', '51.218.147/0001-93', '06.977.751/0001-49','06.977.745/0001-91']


In [ ]:
filtrados = df[df['CNPJ_CIA'].isin(lista_cnpjs)]

In [ ]:
filtrados

In [ ]:
filtrados = filtrados[filtrados.TP_MERC.str.contains('BOLSA', na=False)]

In [ ]:
filtrados

In [ ]:
list(filtrados['CD_CVM'])

## 2. Extração dos demonstrativos trimestrais das empresas 

In [ ]:
import io
import zipfile

In [ ]:
link = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2021.zip'
arquivo = 'itr_cia_aberta_DRE_con_2021.csv'

In [ ]:
# busca arquivo zip no site 
arquivo_zip = requests.get(link)
# abre o arquivo zip para uso 
zf = zipfile.ZipFile(io.BytesIO(arquivo_zip.content))

In [ ]:
zf

In [ ]:
# abrindo o arquivo itr_cia_aberta_DRE_con_2021.csv que está dentro do zip
DRE = zf.open(arquivo)

In [ ]:
# para saber qual o tipo do objeto use a função "type(objeto)"
type(DRE)

In [ ]:
# extraindo o conteudo do arquivo para uma lista
linhas = DRE.readlines()

In [ ]:
# a lista "linhas" possui todas as linhas do arquivo DRE
linhas
# porém é necessário extrair os espaços extras usando a função strip e usar decode para exibição correta dos caracteres da lingua portuguesa
lines = [i.strip().decode('ISO-8859-1') for i in linhas ]

In [ ]:
lines

In [ ]:
# transformando cada linha em line em listas usando o ; para "quebrar" as informações
lines = [i.split(';') for i in lines ]

In [ ]:
# Gerando um DataFrame usando a função DataFrame do pandas
df = pd.DataFrame(lines[1:],columns = lines[0])

In [ ]:
df.head()

In [ ]:
# os dados dos demonstrativos trazem o código CVM com um zero acrescido a esquerda
df[df['CD_CVM']=='019909']

In [ ]:
df.dtypes

## 3. Criação da rotina de extração

In [ ]:
df.DT_FIM_EXERC.unique()

In [ ]:
df.DT_REFER.unique()

### Inicio da construção do loop

In [ ]:
# Vamos criar um loop que vai capturar varios demonstrativos de uma vez, e depois filtrar empresas específicas
# Quando isso for terminado, ele vai gerar um excel por empresa, onde cada aba será um demonstrativo diferente
# O usuário terá apenas de passar uma lista de empresas e uma lista de demonstrativos
demonstrativos = ['DFC_MD','DFC_MI','BPA','DRE']

In [ ]:
!pip install xlsxwriter

In [ ]:
import xlsxwriter
import zipfile
import io
import time

# salvando o tempo de inicio da execucao
start_time = time.time()

demonstrativos = ['DFC_MD','DFC_MI','BPA','BPP','DRE']

empresas = list(filtrados['CD_CVM'])
# empresas.append('9512')

link = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2021.zip'

lista_listas = []

a = 0

for j in empresas:

  lista_df = []

  for k in demonstrativos:

    # busca arquivo zip no site 
    arquivo_zip = requests.get(link)

    # abre o arquivo zip para uso 
    zf = zipfile.ZipFile(io.BytesIO(arquivo_zip.content))

    arquivo = 'itr_cia_aberta_' + str(k) + '_con_2021.csv'

    # abrindo o arquivo itr_cia_aberta_*_con_2021.csv que está dentro do zip
    dados = zf.open(arquivo)

    # extraindo o conteudo do arquivo para uma lista
    linhas = dados.readlines()

    # porém é necessário extrair os espaços extras usando a função strip e usar decode para exibição correta dos caracteres da lingua portuguesa
    lines = [i.strip().decode('ISO-8859-1') for i in linhas ]

    # transformando cada linha em line em listas usando o ; para "quebrar" as informações
    lines = [i.split(';') for i in lines ]

    # Gerando um DataFrame usando a função DataFrame do pandas
    df = pd.DataFrame(lines[1:],columns = lines[0])

    # Preparando colunas para serem usadas em cáculos
    # df.dtypes
    # CNPJ_CIA         object
    # DT_REFER         object
    # VERSAO           object
    # DENOM_CIA        object
    # CD_CVM           object
    # GRUPO_DFP        object
    # MOEDA            object
    # ESCALA_MOEDA     object
    # ORDEM_EXERC      object
    # DT_INI_EXERC     object
    # DT_FIM_EXERC     object
    # CD_CONTA         object
    # DS_CONTA         object
    # VL_CONTA         object << para usar em cáculos matemáticos é necessário gerar como numérico
    # ST_CONTA_FIXA    object
    # dtype: object

    df['VL_AJUSTADO'] = pd.to_numeric(df['VL_CONTA'])

    # filtrando os dados do data frame referente a empresa "j" do loop 
    # filtro = df[df['CD_CVM'] == '0' + str(j)]
    # o metodo zfill substitui a concatenação do 0 a esquerda da string com a vantagem de preencher conm tantos zeros quantos 
    # sejam necessários a esquerda até que a string tenha o tamanho indicado na chamada do metodo. Nesse caso (6)
    filtro = df[df['CD_CVM'] == str(j).zfill(6)]
    # acrescentando o data frame a lista
    lista_df.append(filtro)
    # gerando log
    print(f'Trabalhando com a empresa {str(j)} e seu demonstrativo {str(k)}. As dimensões são {filtro.shape}')

  lista_listas.append(lista_df)

  # definindo o arquivo excel usando a bibliotece xlsxwriter. um arquivo excel por empresa
  writer = pd.ExcelWriter(f'Demonstrativos_Empresa_{str(j)}.xlsx', engine='xlsxwriter')
  
  # definindo as abas do excel. uma aba para cada demonstrativo
  lista_listas[a][0].to_excel(writer, sheet_name='DFC_MD', encoding='ISO-8859-1')
  lista_listas[a][1].to_excel(writer, sheet_name='DFC_MI', encoding='ISO-8859-1')
  lista_listas[a][2].to_excel(writer, sheet_name='BPA', encoding='ISO-8859-1')
  lista_listas[a][3].to_excel(writer, sheet_name='BPP', encoding='ISO-8859-1')
  lista_listas[a][4].to_excel(writer, sheet_name='DRE', encoding='ISO-8859-1')

  a += 1

  print(f'Arquivo excel com dados da empresa {str(j)} foi exportado. \n')

  # Salvar o arquivo excel
  writer.save()

print ("O tempo de execução desse programa foi de %s --- segundos" %(time.time() - start_time))




In [ ]:
lista_listas[0][1]

In [ ]:
empresas = list(filtrados['CD_CVM'])
empresas

#4. Transformando os dados

### 4.1 Instalando e importando bibliotecas

In [ ]:
!pip install Plotly
!pip install investpy

In [ ]:
import plotly
import investpy
import os
import numpy as np
import pandas as pd

### 4.2. Tabela DRE

In [ ]:
caminho = os.getcwd()

In [ ]:
arquivos = os.listdir(caminho)

In [ ]:
arquivo_xls = [f for f in arquivos if f[-1] == 'x'] # 04.03.02 aos 5:36 minutos

In [ ]:
arquivo_xls

In [ ]:
dre = pd.DataFrame()

In [ ]:
for f in arquivo_xls:
  drexls = pd.read_excel(f, sheet_name='DRE')
  dre = dre.append(drexls)

In [ ]:
dre

In [ ]:
lista_de_empresas = dre['DENOM_CIA'].unique()

In [ ]:
lista_de_empresas

In [ ]:
lista_de_empresas = lista_de_empresas.transpose()

In [ ]:
lista_de_empresas

In [ ]:
lista_de_empresas = lista_de_empresas.tolist()

In [ ]:
lista_de_empresas

In [ ]:
tickers = ['IGTA3','MULT3','BRPR3','BRML3','ALSO3']

In [ ]:
n_empresas = len(lista_de_empresas)

In [ ]:
n_empresas 

In [ ]:
lista_de_contas = dre['DS_CONTA'].unique()
lista_de_contas = lista_de_contas.tolist()

In [ ]:
lista_de_contas

In [ ]:
# organizar a tabela como tabela dinâmica (pivot table), onde as datas sejam labels de coluna
dre = pd.pivot_table(dre, index=['DENOM_CIA','DS_CONTA'], columns=['DT_INI_EXERC','DT_FIM_EXERC'], values=['VL_AJUSTADO'])

In [ ]:
dre

In [ ]:
dre.loc['BR PROPERTIES S.A.',:].loc['Resultado Bruto']

In [ ]:
type(dre)

In [ ]:
dre.loc['BR PROPERTIES S.A.',:]

### 4.3 Tabela Balanço Patrimonial Ativo(BPA) e Passivo(BPP)

In [ ]:
bpa = pd.DataFrame()

for f in arquivo_xls:
  bpa_xls = pd.read_excel(f, sheet_name='BPA')
  bpa = bpa.append(bpa_xls)

In [ ]:
bpa

In [ ]:
bpa = pd.pivot_table(bpa, index=['DENOM_CIA', 'DS_CONTA'], columns=['DT_FIM_EXERC'], values=['VL_AJUSTADO'])
bpa

In [ ]:
bpp = pd.DataFrame()
for f in arquivo_xls:
  bpp_xls = pd.read_excel(f, sheet_name='BPP')
  bpp = bpp.append(bpp_xls)
  bpp

In [ ]:
# bpp = pd.pivot_table(bpp, index=['DENOM_CIA','DS_CONTA'], columns=['DT_FIM_EXERC'], values=['VL_AJUSTADO'])
bpp = pd.pivot_table(bpp, index=['DENOM_CIA','DS_CONTA','CD_CONTA'], columns=['DT_REFER','DT_FIM_EXERC'],values=['VL_AJUSTADO'])

# 5. Indicadores Fundamentalistas

## 5.1 Margens

In [ ]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [ ]:
type(dre)

In [ ]:
dre

In [ ]:
dre.loc['BR MALLS PARTICIPACOES S.A.',:]

In [ ]:
dre.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Resultado Bruto']

In [ ]:
dre.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Resultado Bruto'].iloc[-1]

In [ ]:
dre.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1]
### margem bruta BR MALLS PARTICIPACOES = 268207.0 / 291742.0 = 0.919329407

In [ ]:
for i in range(0,n_empresas):
  calculo_margem = ((dre.loc[lista_de_empresas[i],:].loc['Resultado Bruto'].iloc[-1]) / (dre.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1]))
  print(f'A margem bruta da empresa {lista_de_empresas[i]} eh {calculo_margem}')

In [ ]:
import plotly.graph_objects as go

In [ ]:
# Margem Bruta
margem_bruta_shopping = pd.DataFrame()
for i in range(0,n_empresas):
  calculo_margem = pd.Series(((dre.loc[lista_de_empresas[i],:].loc['Resultado Bruto'].iloc[-1]) / (dre.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1])))
  margem_bruta_shopping = pd.concat([margem_bruta_shopping,calculo_margem], axis=1)
margem_bruta_shopping.columns = lista_de_empresas
fig = go.Figure(go.Bar(x=margem_bruta_shopping.iloc[-1], y=tickers, orientation='h'))
fig.show()


In [ ]:
margem_bruta_shopping

In [ ]:
margem_bruta_shopping.iloc[0] # nesse caso é igual a iloc[-1] pois só temos uma linha no dataframe

In [ ]:
# Margem Líquida
dre.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Lucro/Prejuízo Consolidado do Período'].iloc[-1]
dre.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Lucro/Prejuízo Consolidado do Período'].iloc[-1]

In [ ]:
dre.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Resultado Bruto'].iloc[-1]

In [ ]:
dre.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1]

In [ ]:
pd.Series((dre.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Lucro/Prejuízo Consolidado do Período'].iloc[-1])/(dre.loc['BR MALLS PARTICIPACOES S.A.',:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1]))

In [ ]:
def margem_liquida():
  margem_liq_shopping = pd.DataFrame()
  for i in range(0, n_empresas):
    calculo_margem_liq = pd.Series((dre.loc[lista_de_empresas[i],:].loc['Lucro/Prejuízo Consolidado do Período'].iloc[-1])/(dre.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1]))
    margem_liq_shopping = pd.concat([margem_liq_shopping,calculo_margem_liq], axis=1)
  margem_liq_shopping.columns = lista_de_empresas
  fig = go.Figure(go.Bar(x=margem_liq_shopping.iloc[-1],y=tickers,orientation='h'))
  fig.update_layout(title_text='Margem Líquida')
  fig.show()


In [ ]:
margem_liquida()